In [1]:
import json
import pandas as pd

from itertools import product

In [2]:
year = 2019
top = 20

with open(f"../../../Data/results/processed/Serie_A_{year}_games.json") as f: real_data = json.load(f)
with open(f"../real_data/Poisson_with_players_model_data_{year}.json") as f: model_data = json.load(f)
players_map = {idx: cod for idx, cod in enumerate(model_data['players'])}
players_map = pd.Series(players_map)

results = pd.read_csv(f"../results/results_Poisson_with_players_{year}.csv")
results = results[results['variable'].str.contains('skills')]
results['player_index'] = results['variable'].str.extract(r'skills\[(\d+)\]').astype(int)
results['player_cod'] = results['player_index'].map(players_map)
top_players_df = results.sort_values(by="mean", ascending=False).head(top)
top_players = top_players_df['player_cod'].tolist()
top_players = {cod: {'name': list(), 'clubs': list()} for cod in top_players}

for game in real_data:
    players = real_data[game]['Players']
    for cod, player in product(top_players, players):
        if cod in player[0]:
            name = player[0]
            name = name \
                .replace(f'RP{cod}', '') \
                .replace(f'TP{cod}', '') \
                .replace(f'R(g)P{cod}', '') \
                .replace(f'T(g)P{cod}', '')
            
            while name[0].isdigit(): name = name[1:]
            name = name.strip()
            top_players[cod]['name'].append(name)
            top_players[cod]['clubs'].append(player[1])

top_players = {k: {'name': list(set(v['name'])), 'clubs': list(set(v['clubs']))} for k, v in top_players.items()}
top_players

{'441648': {'name': ['Rodrygo Rodrygo Silva de Goes'],
  'clubs': ['Santos / SP']},
 '291617': {'name': ['Richard Richard de Oliveira Costa'],
  'clubs': ['Ceará / CE']},
 '183094': {'name': ['Romulo Romulo Borges Monteiro'],
  'clubs': ['Grêmio / RS']},
 '633582': {'name': ['Derlis Gon ... Derlis Alberto Gonza ...',
   'Derlis Gon ... Derlis Alberto Gonza...',
   'Derlis Gon ...Derlis Alberto Gonza ...'],
  'clubs': ['Santos / SP']},
 '302795': {'name': ['Erik Erik Nascimento de Lima'],
  'clubs': ['Botafogo / RJ']},
 '165065': {'name': ['ParáMarcos Rogerio Ricci ...',
   'Pará Marcos Rogerio Ricci ...',
   'Pará Marcos Rogerio Ricci...'],
  'clubs': ['Flamengo / RJ', 'Santos / SP']},
 '376734': {'name': ['Clayson Clayson Henrique da  ...',
   'Clayson Clayson Henrique da ...'],
  'clubs': ['Corinthians / SP']},
 '144059': {'name': ['Betão Ebert Willian Amancio',
   'EBERT WILLIAN AMANCIO T 144059'],
  'clubs': ['Avaí / SC']},
 '307245': {'name': ['Vitinho Victor Vinicius Coel ...'],


In [3]:
top_players_df

,variable,mean,median,sd,mad,q5,q95,q2.75,q97.5,player_index,player_cod
302,skills[302],1.314771,1.293365,0.615788,0.644475,0.321851,2.367612,0.220152,2.553855,302,441648
430,skills[430],1.242177,1.213810,0.611623,0.646017,0.293544,2.318249,0.194893,2.520779,430,291617
324,skills[324],1.231262,1.209025,0.586037,0.610003,0.308337,2.219629,0.214017,2.447780,324,183094
25,skills[25],1.222323,1.194695,0.601750,0.618109,0.249643,2.256435,0.154112,2.439223,25,633582
131,skills[131],1.218844,1.192010,0.608668,0.636371,0.263820,2.275284,0.175236,2.460969,131,302795
146,skills[146],1.195632,1.157480,0.608015,0.637342,0.245026,2.244776,0.170461,2.492062,146,165065
237,skills[237],1.189167,1.169080,0.607000,0.637514,0.235602,2.250108,0.135495,2.456952,237,376734
40,skills[40],1.183788,1.161190,0.606338,0.647615,0.252632,2.238615,0.151606,2.450105,40,144059
410,skills[410],1.142932,1.091575,0.629511,0.671303,0.197877,2.245875,0.128621,2.465172,410,307245
86,skills[86],1.140915,1.093730,0.625035,0.673375,0.191320,2.259518,0.105165,2.474588,86,191616
